## Create _P.generosa_ primary gene annotations mapping file

See this [GitHub Issue(https://github.com/RobertsLab/resources/issues/1443).

This notebook will utilize NCBI BLAST and DIAMOND BLAST annotations generated by our [GenSas _P.generosa_ genome annotation](https://robertslab.github.io/sams-notebook/2019/09/28/Genome-Annotation-Pgenerosa_v074-a4-Using-GenSAS.html).

It will compare the two sets of SwissProt ID annotations (SPIDs) to determine lowest E-value and use that entry as the representative entry for a gene. It will then use that canonical list of SPIDs to pull gene names and gene ontology (GO) IDs from UniProt, and create a tab-deltimited annotation mapping file.

### List computer specs

In [1]:
%%bash
echo "TODAY'S DATE"
date
echo "------------"
echo ""
lsb_release -a
echo ""
echo "------------"
echo "HOSTNAME: "
hostname
echo ""
echo "------------"
echo "Computer Specs:"
echo ""
lscpu
echo ""
echo "------------"
echo ""
echo "Memory Specs"
echo ""
free -mh

TODAY'S DATE
Tue 12 Apr 2022 02:02:11 PM PDT
------------

Distributor ID:	Ubuntu
Description:	Ubuntu 20.04.4 LTS
Release:	20.04
Codename:	focal

------------
HOSTNAME: 
computer

------------
Computer Specs:

Architecture:                    x86_64
CPU op-mode(s):                  32-bit, 64-bit
Byte Order:                      Little Endian
Address sizes:                   45 bits physical, 48 bits virtual
CPU(s):                          2
On-line CPU(s) list:             0,1
Thread(s) per core:              1
Core(s) per socket:              1
Socket(s):                       2
NUMA node(s):                    1
Vendor ID:                       GenuineIntel
CPU family:                      6
Model:                           165
Model name:                      Intel(R) Core(TM) i9-10885H CPU @ 2.40GHz
Stepping:                        2
CPU MHz:                         2400.006
BogoMIPS:                        4800.01
Hypervisor vendor:               VMware
Virtualization type:     

No LSB modules are available.


### Set variables
- `%env` indicates a bash variable
- without `%env` is Python variablec

In [2]:
######################################################################
### Set directories
%env data_dir=/home/sam/data/P_generosa/genomes
%env analysis_dir=/home/sam/analyses/20220406-pgen-gene_annotation_mapping
analysis_dir="/home/sam/analyses/20220406-pgen-gene_annotation_mapping"

#####################################################################
### Input files
%env base_url=https://gannet.fish.washington.edu/Atumefaciens/20190928_Pgenerosa_v074.a4_gensas_annotation
%env blast_annotations=Panopea-generosa-vv0.74.a4.5d951a9b74287-blast_functional.tab
%env diamond_annotations=Panopea-generosa-vv0.74.a4.5d951bcf45b4b-diamond_functional.tab

######################################################################
### Output files
# UniProt batch output
%env uniprot_output=20220406-pgen-uniprot_batch-results.txt

# Gene name list for UniProt batch submission
%env spid_list=Panopea-generosa-v1.0.a4-blast-diamond-functional-SPIDs.txt

%env blast_diamond_cat=Panopea-generosa-v1.0.a4-blast-diamond-functional.tab

# Parsed UniProt
%env parsed_uniprot=20220406-pgen-accession-gene_name-gene_description-go_ids.csv

# Final output
%env joined_output=20220406-pgen-gene-gene_name-gene_description-go_ids.csv

######################################################################

### Programs

# UniProt batch submission/retrieval script
%env uniprot_mapping_script=/home/sam/programs/uniprot_mapping.pl

env: data_dir=/home/sam/data/P_generosa/genomes
env: analysis_dir=/home/sam/analyses/20220406-pgen-gene_annotation_mapping
env: base_url=https://gannet.fish.washington.edu/Atumefaciens/20190928_Pgenerosa_v074.a4_gensas_annotation
env: blast_annotations=Panopea-generosa-vv0.74.a4.5d951a9b74287-blast_functional.tab
env: diamond_annotations=Panopea-generosa-vv0.74.a4.5d951bcf45b4b-diamond_functional.tab
env: uniprot_output=20220406-pgen-uniprot_batch-results.txt
env: spid_list=Panopea-generosa-v1.0.a4-blast-diamond-functional-SPIDs.txt
env: blast_diamond_cat=Panopea-generosa-v1.0.a4-blast-diamond-functional.tab
env: parsed_uniprot=20220406-pgen-accession-gene_name-gene_description-go_ids.csv
env: joined_output=20220406-pgen-gene-gene_name-gene_description-go_ids.csv
env: uniprot_mapping_script=/home/sam/programs/uniprot_mapping.pl


### Make input/output directories

In [3]:
%%bash
# If directories don't exist, make them
mkdir --parents "${data_dir}" "${analysis_dir}"

### Download and inspect annotation files

`--quiet`: Prevents `wget` output from overwhelming Jupyter Notebook

`--continue`: If download was previously initiated, will continue where leftoff and will not create a second file if one already exists.

In [4]:
%%bash
cd "${data_dir}"

wget --quiet --continue ${base_url}/${blast_annotations}
wget --quiet --continue ${base_url}/${diamond_annotations}

ls -ltrh

head -n 25 *.tab

total 2.6G
-rw-rw-r-- 1 sam sam 1.5M Oct  3  2019 Panopea-generosa-vv0.74.a4.5d951a9b74287-blast_functional.tab
-rw-rw-r-- 1 sam sam 1.3M Oct  3  2019 Panopea-generosa-vv0.74.a4.5d951bcf45b4b-diamond_functional.tab
-rwxr-xr-x 1 sam sam 914M Nov  5  2019 Panopea-generosa-v1.0.fasta
-rw-rw-r-- 1 sam sam 454M Mar 19 07:58 Panopea-generosa-v1.0.a4.gff3
-rw-r--r-- 1 sam sam 503M Mar 22 06:48 Panopea-generosa-v1.0.a4_biotype.gff
-rw-r--r-- 1 sam sam 4.8M Mar 24 07:30 Panopea-generosa-v1.0.a4_biotype-trna_strand_converted-no_RNAmmer.bed
-rw-rw-r-- 1 sam sam  658 Mar 25 06:11 Panopea-generosa-v1.0.fa.fai
-rw-rw-r-- 1 sam sam 9.7M Mar 30 11:03 Panopea-generosa-v1.0.a4_biotype.gtf
-rw-rw-r-- 1 sam sam 507M Mar 30 11:43 Panopea-generosa-v1.0.a4_biotype.bed
-rw-rw-r-- 1 sam sam  378 Mar 30 13:20 Panopea-generosa-v1.0.fa.lengths
-rw-rw-r-- 1 sam sam 9.7M Mar 30 13:34 Panopea-generosa-v1.0.a4_biotype.sorted.gtf
-rw-rw-r-- 1 sam sam 996K Mar 30 13:34 Panopea-generosa-v1.0.a4_biotype_non-coding.bed
dr

### Count number of header lines (i.e. beginning with a `#`

In [5]:
%%bash
cd "${data_dir}"

grep -c "^#" "${blast_annotations}" "${diamond_annotations}"

Panopea-generosa-vv0.74.a4.5d951a9b74287-blast_functional.tab:12
Panopea-generosa-vv0.74.a4.5d951bcf45b4b-diamond_functional.tab:12


### Concatenate annotation files

Also modifies mRNA names to generate gene names instead.

- `awk 'NR > 13'`: Skips first 13 header lines
- `sort -k1,1 -k9,9`: Sorts on first field (mRNA name), then on 9th field (e-value)
- `sed 's/^21910-//'`: Removes leading info from each mRNA name, at the beginning of each line (`^`)
- `sed 's/.m0[0-9]//'`: Removes `.m0N` from each mRNA name.
- `awk '!array[$1]++'`: `awk` array that only prints line if it's the first occurrence of gene name (first field; `$1` (i.e. no duplicates)

In [6]:
%%bash
cd "${data_dir}"

for file in ${blast_annotations} ${diamond_annotations}
do
    awk 'NR > 13' ${file}
done \
| sort -k1,1 -k9,9 \
| sed 's/^21910-//' \
| sed 's/.m0[0-9]//' \
| awk '!array[$1]++'\
>> "${blast_diamond_cat}"

wc -l "${blast_diamond_cat}"

head -n 25 "${blast_diamond_cat}"

29353 Panopea-generosa-v1.0.a4-blast-diamond-functional.tab
PGEN_.00g000010	121	229	165	Q86IC9	sp|Q86IC9|CAMT1_DICDI	11	122	8.93e-14	35.652	115
PGEN_.00g000020	147	467	968	P04177	sp|P04177|TY3H_RAT	20	339	3.47e-127	55.140	321
PGEN_.00g000050	566	722	180	Q8L840	sp|Q8L840|RQL4A_ARATH	2	167	2.4e-12	35.1	168
PGEN_.00g000060	1957	2106	129	Q61043	sp|Q61043|NIN_MOUSE	31	184	1.7e-06	26.0	154
PGEN_.00g000080	268	322	152	A1E2V0	sp|A1E2V0|BIRC3_CANLF	163	220	3.91e-10	53.448	58
PGEN_.00g000090	199	327	161	P34456	sp|P34456|YMD2_CAEEL	7	134	7.52e-12	26.357	129
PGEN_.00g000120	6	49	118	P34457	sp|P34457|YMD3_CAEEL	90	133	3.2e-05	47.7	44
PGEN_.00g000210	18	200	263	O00463	sp|O00463|TRAF5_HUMAN	5	191	2.24e-25	34.921	189
PGEN_.00g000230	48	155	287	Q00945	sp|Q00945|CONO_LYMST	31	134	1.59e-32	50.000	108
PGEN_.00g000240	4	585	1144	Q5SWK7	sp|Q5SWK7|RN145_MOUSE	13	592	1.2e-123	40.1	591
PGEN_.00g000280	4	153	210	Q8ZXT3	sp|Q8ZXT3|Y1111_PYRAE	853	1012	1.10e-17	38.750	160
PGEN_.00g000300	159	347	480	Q5REG4	sp|Q5RE

### Create list os SwissProt IDs

In [7]:
%%bash
cd "${analysis_dir}"

awk '{print $5}' "${data_dir}"/"${blast_diamond_cat}" > "${spid_list}"

wc -l "${spid_list}"

head "${spid_list}"

29353 Panopea-generosa-v1.0.a4-blast-diamond-functional-SPIDs.txt
Q86IC9
P04177
Q8L840
Q61043
A1E2V0
P34456
P34457
O00463
Q00945
Q5SWK7


### Batch submission/retrieval to/from UniProt

Perl script obtained from UniProt: https://www.uniprot.org/help/api_batch_retrieval

Modified to accept file with list of IDs and to map SPID to UniProt Accession

In [8]:
%%bash
# Print script for viewing
cat "${uniprot_mapping_script}"

use strict;
use warnings;
use LWP::UserAgent;

my $list = $ARGV[0]; # File containg list of UniProt identifiers.

my $base = 'https://www.uniprot.org';
my $tool = 'uploadlists';

my $contact = 'samwhite@uw.edu'; # Please set a contact email address here to help us debug in case of problems (see https://www.uniprot.org/help/privacy).
my $agent = LWP::UserAgent->new(agent => "libwww-perl $contact");
push @{$agent->requests_redirectable}, 'POST';

my $response = $agent->post("$base/$tool/",
                            [ 'file' => [$list],
                              'format' => 'txt',
                              'from' => 'SWISSPROT',
                              'to' => 'ACC',
                            ],
                            'Content_Type' => 'form-data');

while (my $wait = $response->header('Retry-After')) {
  print STDERR "Waiting ($wait)...\n";
  sleep $wait;
  $response = $agent->get($response->base);
}

$response->is_success ?
  print $response->content :
  die 'Fail

In [9]:
%%bash
cd "${analysis_dir}"

# Run UniProt Prel mapping script and time how long it takes
time \
perl "${uniprot_mapping_script}" "${spid_list}" > "${uniprot_output}" 2> "${uniprot_output}".stderr

ls -ltrh

echo ""
echo ""
echo "--------------------------------------------------"
echo ""
echo "Line count:"

wc -l "${uniprot_output}"

echo "--------------------------------------------------"

total 138M
-rw-rw-r-- 1 sam sam 202K Apr 12 14:03 Panopea-generosa-v1.0.a4-blast-diamond-functional-SPIDs.txt
-rw-rw-r-- 1 sam sam    0 Apr 12 14:04 20220406-pgen-uniprot_batch-results.txt.stderr
-rw-rw-r-- 1 sam sam 138M Apr 12 14:04 20220406-pgen-uniprot_batch-results.txt


--------------------------------------------------

Line count:
2850419 20220406-pgen-uniprot_batch-results.txt
--------------------------------------------------



real	0m44.314s
user	0m3.264s
sys	0m3.967s


### Check mapping output

Counting Accession lines (beginning with `AC`) should show a _lower_ count than the number of SwissProt IDs submitted, as UniProt automatically removes duplicates upon submission.

In [10]:
%%bash
cd "${analysis_dir}"

head -n 30 "${uniprot_output}"

echo ""

echo "----------------------------------------------------"

echo ""

echo "Number of accessions:"

echo ""

grep -c "^AC" "${uniprot_output}"

ID   CAMT1_DICDI             Reviewed;         230 AA.
AC   Q86IC9; Q552T5;
DT   05-MAY-2009, integrated into UniProtKB/Swiss-Prot.
DT   01-JUN-2003, sequence version 1.
DT   23-FEB-2022, entry version 92.
DE   RecName: Full=Probable caffeoyl-CoA O-methyltransferase 1;
DE            EC=2.1.1.104;
DE   AltName: Full=O-methyltransferase 5;
GN   Name=omt5; ORFNames=DDB_G0275499;
OS   Dictyostelium discoideum (Slime mold).
OC   Eukaryota; Amoebozoa; Evosea; Eumycetozoa; Dictyostelia; Dictyosteliales;
OC   Dictyosteliaceae; Dictyostelium.
OX   NCBI_TaxID=44689;
RN   [1]
RP   NUCLEOTIDE SEQUENCE [LARGE SCALE GENOMIC DNA].
RC   STRAIN=AX4;
RX   PubMed=12097910; DOI=10.1038/nature00847;
RA   Gloeckner G., Eichinger L., Szafranski K., Pachebat J.A., Bankier A.T.,
RA   Dear P.H., Lehmann R., Baumgart C., Parra G., Abril J.F., Guigo R.,
RA   Kumpf K., Tunggal B., Cox E.C., Quail M.A., Platzer M., Rosenthal A.,
RA   Noegel A.A.;
RT   "Sequence and analysis of chromosome 2 of Dictyostelium discoide

### Parse the stuff we want

- UniProt accession

- Gene name/abbreviation

- Gene description

- GO IDs

#### Check DE descriptor lines to decide pattern matching

Checks lines beginning with `DE` to identify values in the 2nd field with `Name` in them.

Identifies unique values. This will determine how to parse properly after this.

In [11]:
%%bash
cd "${analysis_dir}"

grep "^DE" "${uniprot_output}" | awk '$2 ~ /Name/ { print $2 }' | sort -u

AltName:
RecName:


In [3]:
%%bash
cd "${analysis_dir}"

# Loop through UniProt records
time \
while read -r line
do
  # Get record line descriptor
  descriptor=$(echo "${line}" | awk '{print $1}')

  # Capture second field for evaluation
  go_line=$(echo "${line}" | awk '{print $2}')

  # Append GO IDs to array
  if [[ "${go_line}" == "GO;" ]]; then
    go_id=$(echo "${line}" | awk '{print $3}')
    go_ids_array+=("${go_id}")
  elif [[ "${go_line}" == "GeneID;" ]]; then
    # Uses sed to strip trailing semi-colon
    gene_id=$(echo "${line}" | awk '{print $3}' | sed 's/;$//')
  fi

  # Get gene description
  if [[ "${descriptor}" == "DE" ]] && [[ "${go_line}" == "RecName:" ]]; then
    # Uses sed to strip trailing spaces at end of line and remove commas
    gene_description=$(echo "${line}" | awk -F "[={]" '{print $2}' | sed 's/[[:blank:]]*$//' | sed 's/,//g' | sed 's/;$//')

  # Get alternate name
  elif [[ "${descriptor}" == "DE" ]] && [[ "${go_line}" == "AltName:" ]]; then
    # Uses sed to strip trailing spaces at end of line and remove commas
    alt_gene_description=$(echo "${line}" | awk -F "[={]" '{print $2}' | sed 's/[[:blank:]]*$//' | sed 's/,//g' | sed 's/;$//')

  # Get gene name
  elif [[ "${descriptor}" == "GN"  ]] && [[ $(echo "${line}" | awk -F "=" '{print $1}') == "GN   Name" ]]; then
    # Uses sed to strip trailing spaces at end of line
    gene=$(echo "${line}" | awk -F 'Name=|{|;' '{print $2}' | sed 's/[[:blank:]]*$//')

  # Get UniProt accession
  elif [[ "${descriptor}" == "AC" ]]; then
    # awk removes "AC" denotation
    # sed removes all spaces
    # sed removes trailing semi-colon
    accession=$(echo "${line}" | awk '{$1="";print $0}' | sed 's/[[:space:]]*//g' | sed 's/;$//')

  # Identify beginning on new record
  elif [[ "${descriptor}" == "//" ]]; then

    # Prints other comma-separated variables, then GOID1;GOID2;GOIDn
    # IFS prevents spaces from being added between GO IDs
    # sed removes ";" after final GO ID
    (IFS=; printf "%s\t%s\t%s\t%s\t%s\t%s\n" "${accession}" "${gene_id}" "${gene}" "${gene_description}" "${alt_gene_description}" "${go_ids_array[*]}" | sed 's/;$//')

    # Re-initialize variables
    accession=""  
    descriptor=""
    gene=""
    gene_description
    gene_id=""
    go_id=""
    go_ids_array=()
  fi


done < "${uniprot_output}" >> "${parsed_uniprot}"

Process is interrupted.


### Inspect parsed UniProt file

In [4]:
%%bash
cd "${analysis_dir}"

head -n 25 "${parsed_uniprot}"

Q86IC9;Q552T5	8620183	omt5; ORFNames=DDB_G0275499;	Probable caffeoyl-CoA O-methyltransferase 1;	O-methyltransferase 5;	GO:0042409;GO:0046872;GO:0008757;GO:0032259
P04177	25085	Th;	Tyrosine 3-monooxygenase;	Tyrosine 3-hydroxylase;	GO:0030424;GO:0005737;GO:0009898;GO:0031410;GO:0030659;GO:0005829;GO:0030425;GO:0033162;GO:0005739;GO:0043005;GO:0043025;GO:0005634;GO:0043204;GO:0048471;GO:0005790;GO:0008021;GO:0043195;GO:0016597;GO:0035240;GO:0019899;GO:0008199;GO:0008198;GO:0042802;GO:0004497;GO:0019825;GO:0019904;GO:0034617;GO:0004511;GO:0015842;GO:0009887;GO:0042423;GO:0071312;GO:0071333;GO:0071363;GO:0071287;GO:0071316;GO:0071466;GO:0021987;GO:0042745;GO:0050890;GO:0042416;GO:0006585;GO:0042755;GO:0048596;GO:0042418;GO:0042462;GO:0006631;GO:0016137;GO:0007507;GO:1990384;GO:0033076;GO:0007612;GO:0007626;GO:0007617;GO:0007613;GO:0010259;GO:0042136;GO:0042421;GO:0018963;GO:0052314;GO:0008016;GO:0014823;GO:0001975;GO:0051412;GO:0051602;GO:0032355;GO:0045471;GO:0045472;GO:0070848;GO:0009635;